In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM

def get_elocutions(path):
    ''' 全ての台詞を取り出す処理 '''
    with open(path, encoding='utf-8') as f:
        ls = f.readlines()
        #print(ls)
        start_elocution = True
        elocutions = []
        for i in range(len(ls)):
            if start_elocution:
                elocution = [ls[i]]
                start_elocution = False
            elif ls[i]=='\n':
                elocution.append('*')
                elocutions.append("".join(elocution))
                start_elocution = True
            else:
                elocution.append(ls[i])
                start_elocution = False
    return elocutions

def return_processed(X, T, lower_bound=10):
    ''' 入力Xに対し、lower_bound < 文字の長さ < T の台詞のみ取る処理 '''
    if len(X)>T or len(X)<lower_bound:
        return None
    else:
        return X.ljust(T, '*')

def preprocessing(corpus, T=80):
    ''' corpusの台詞X毎に return_processed をかける '''
    preprocessed_corpus = []
    for X in corpus:
        X_ = return_processed(X, T)
        if X_ is not None:
            preprocessed_corpus.append(X_)
    return preprocessed_corpus

def get_corpus(path, T=80):
    ''' 前処理したコーパスを返す '''
    corpus = get_elocutions(path)
    return preprocessing(corpus, T=T)

class Generator(tf.keras.Model): # モデル設計
    def sample_from(self, start_string, num_string=100):
        ''' This function is derived from the function `generate_text` in
        https://www.tensorflow.org/tutorials/text/text_generation 
        which is licenced under Apache 2.0 License. '''
        input_nums = tf.expand_dims([char2idx[c] for c in start_string], 0)
        text_generated = []
        model.reset_states()
        for i in range(num_string):
            if char2idx['*'] in input_nums.numpy():
                break
            #print(input_nums)
            z = self(input_nums)[:,-1,:]
            predicted_id = tf.random.categorical(z, num_samples=1) # このサンプリングは logit を指定して softmax からサンプリング
            text_generated.append(tf.squeeze(predicted_id).numpy())
            input_nums = predicted_id
        print(start_string+"".join([idx2char[g] for g in text_generated]))
        
def get_data_dicts():
    ''' This function is derived from functions prepareing `dataset` object in
        https://www.tensorflow.org/tutorials/text/text_generation 
        which is licenced under Apache 2.0 License. '''
    path = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
    corpus = get_corpus(path); chars = sorted(set(str("".join(corpus))))
    char2idx = {u:i for i, u in enumerate(chars)}; idx2char = np.array(chars)
    corpus_num = np.array([[char2idx[c] for c in n] for n in corpus])
    D = tf.data.Dataset.from_tensor_slices(corpus_num)
    f = lambda n: (n[:-1], n[1:])
    D = D.map(f)
    return D, chars, char2idx, idx2char
    
def loss_sum(y, z):
    return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y, z, from_logits=True)) # z=model(x) は logit

def update(X, Y, model, optimizer): # 学習ステップ
    with tf.GradientTape() as tape:
        Z = model(X) 
        loss_value = loss_sum(Y, Z) 
    grads = tape.gradient(loss_value, model.trainable_variables) 
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss_value

## 4-3. 時系列データと再帰的な構造
まずは前節のシェイクスピアデータを読み込みます：

In [2]:
D, chars, char2idx, idx2char = get_data_dicts()

過去の状態を考慮に入れるニューラルネットワークとして、

$$
q_\theta(n_{t+1} \mid [n_0, n_1, \dots, n_t])
$$

の形式のモデルはグラフで書くと、

![alt](rnn.jpg)

のように、時間方向にも矢印が伸びるようなネットワークで表せます。このような **時間方向に自己相互作用する** ニューラルネットワークを **リカレントニューラルネットワーク(RNN)** と呼びます。

### 素朴なRNN
まずは#で作った `class Markov` を拡張したモデル `MyRNN` を考えてみましょう：

![alt](rnn2.jpg)

以前までは2層目は `Dense` でしたが、そこを `SimpleRNN` というのに変更しました。この層は毎時刻 $t$ の出力ベクトル ${\bf h}_t$ が次の時刻に（内部で）引き継がれるようになっています。式で書くと以下のようになります：

$$
\left\{ \begin{array}{ll}
{\bf e}_t={\color{red}{l_{emb}}}(n_t) & \color{red}{\text{Embedding}}
\\
{\bf h}_t = {\color{red}{f_{rnn}}}({\bf e}_t)=\tanh({\color{red}{W_{he}}}{\bf e}_t + {\color{red}{W_{hh}}} {\bf h}_{t-1} + {\color{red}{b_h}}) & \color{red}{\text{SimpleRNN}}
\\
{\bf z}_t = {\color{red}{l_z}}({\bf h}_t) = {\color{red}{W_z}} {\bf h}_t + {\color{red}{b_z}} & \color{red}{\text{Dense}}
\\
q_{\color{red}{\theta}}(n\mid [n_0, n_1, \dots, n_t]) = [{\bf \color{blue}{\sigma}}({\bf z}_t)]_{n\text{-th component}} & \color{blue}{\text{Softmax}}
\end{array} \right.
$$

このような時系列の処理を、tensorflowでは



In [3]:
class MyRNN(Generator): # モデル設計
    ''' This model definition is derived from the model defined in
        https://www.tensorflow.org/tutorials/text/text_generation 
        which is licenced under Apache 2.0 License. '''
    def __init__(self, emb_dim, hidden_dim, batch_size, 
                 RNN_layer=SimpleRNN):
        super(MyRNN, self).__init__()
        self.l_emb= tf.keras.layers.Embedding(len(chars), emb_dim)
        self.f_rnn = RNN_layer(hidden_dim,
                        return_sequences=True,
                        stateful=True, # ここを True にすると、状態が保持される
                        recurrent_initializer='glorot_uniform')
        self.l_z = tf.keras.layers.Dense(units=len(chars))
        self.build(tf.TensorShape([batch_size, None])) # 上で stateful=True の場合必要
    def call(self, nt):
        e = self.l_emb(nt)
        h = self.f_rnn(e)
        z = self.l_z(h)
        return z

のように、内部状態 ${\bf h}_t$ の引き継ぎ処理を明示的に書くことなく、あたかも ${n \to {\bf e} \to {\bf h} \to {\bf z}}$ の順伝搬かのようにして書くことで実装できます。
> このようなリカレント層を自分でカスタマイズしたい場合は親玉？の [`tf.keras.layers.RNN`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RNN) を継承したクラスを書くのが良いようですが、今回は深入りしないことにします。

tensorflowでのRNNは、`tf.keras.layers.RNN`のクラスで実装する限り、時系列データの長さ $T$ を揃えておいて、`numpy`のshapeの意味で
- (batchisize, $T$, dim$_{emb}$)

のようなデータ形式であれば勝手にデータのテンソルの二成分目を時系列だと認識して処理してくれるようになっています。

#### 実装(バッチサイズ)に関する注意
`__init__()` の最後の行の処理：`self.build(tf.TensorShape([batch_size, None]))`について注意です。**この実装はモデルを実際にオブジェクトとして作る段階でバッチサイズを指定する仕様になっています** 。これは`RNN_layer(stateful=True)` としていることから外せないようです。ここでstatefulとは ${\bf h}_t$ のような時間方向のベクトルをミニバッチでSGDしてゆく際に初期化せず、そのまま使い続けるというオプションです。おそらく、バッチサイズ分の状態ベクトル ${\bf h}_t$ を保持するためのメモリを確保する処理が必要なため`self.build(tf.TensorShape([batch_size, None]))`の宣言が必要なのだと思われます。このいかにも面倒なオプション
`RNN_layer(stateful=True)`をここで採用する理由は二つです：
- 毎回初期化(`stateful=False`)するとやや処理速度が落ちる
- `stateful=True` のほうが結果が良い

１つ目の理由は自明ですが、２つ目は何故なのか、よくわかりません。どなたか分かったら教えて下さい。ともかく、ここでの実装ではモデルを作る際に指定したバッチサイズを用いて **のみ** 文章生成が行える様になっているため、バッチサイズを例えば `32` に指定すると、学習後も 32文 を生成することしかできません。これでもいいのですが、ここでは[tensorflowチュートリアル](https://www.tensorflow.org/tutorials/text/text_generation?hl=ja)のやり方を参考に、
1. 一旦モデルのパラメータ $\theta$ を適当なファイルに保存する：`model.save_weights('filename')`
2. モデルを使う際は使いたいバッチサイズで新たなモデルを作って、1で保存したパラメータを読み込む：`model.load_weights('filename')`

という方式を採ることにします。


#### SimpleRNNで訓練する
上の注意に気をつけながら訓練させてみます。またここでは今まで訓練ステップ関数 `update` を定義する際につけていた `@tf.function` のデコレータをやめて、毎回 `tf_update = tf.function(update)` を宣言する仕様にしています。これは何度も同じ環境でグラフ化した関数を使う際にはグラフを作り直さないといけないらしいための処置です。デコレータで関数定義を毎回やり直すよりこちらのほうが一行で書けて良いでしょう。

In [4]:
def train_RNN(RNN_layer = SimpleRNN, epoch_size=10):
    tf_update = tf.function(update) # グラフ化（毎回やり直さないとエラーが出る）
    batch_size = 32
    model = MyRNN(emb_dim=256, hidden_dim=1024, batch_size=batch_size, 
                  RNN_layer=RNN_layer) # 
    optimizer=tf.keras.optimizers.Adam()
    loss_averages = []
    ### training
    for epoch in range(epoch_size):
        model.reset_states()
        batch = D.shuffle(5000).batch(batch_size, drop_remainder=True)
        loss_values = []
        for (X,Y) in batch:
            loss_value = tf_update(X, Y, model, optimizer)
            loss_values.append(loss_value)
        loss_averages.append(np.average(loss_values))
    model.save_weights('RNN_test') # stateful で 設計時にバッチサイズを固定した場合、後で使うなら重みを保存する
    return loss_averages

実際の訓練は以下：

In [5]:
%%time
loss_averages = train_RNN(RNN_layer = SimpleRNN, epoch_size=15)

CPU times: user 1min 46s, sys: 9.58 s, total: 1min 56s
Wall time: 1min 3s


#### SimpleRNNでシェイクスピア台詞生成
訓練後にモデルを読み込む関数も作っておきます：

In [6]:
def load_RNN(RNN_layer = SimpleRNN):
    ''' This function is derived from functions for loading saved models defined in
        https://www.tensorflow.org/tutorials/text/text_generation 
        which is licenced under Apache 2.0 License. '''
    model = MyRNN(emb_dim=256, hidden_dim=1024, batch_size=1, RNN_layer=RNN_layer)
    model.load_weights('RNN_test')
    return model

訓練したモデルで台詞生成：

In [7]:
model=load_RNN(RNN_layer = SimpleRNN)
for _ in range(5):
    model.sample_from("A", num_string=100)

ANCANIO:
Ay, go dotien, we is so.
*
AUTHAND: Y ungwer, let's hous.
*
ANTANIE:
That an he shall so thee, epay hi's iffore, as for you?
Musid; the wather our any and sheet,
ANTHAMUS:
Why, how it stand the shall to hour Rime.
*
A TIUK:
What, as he seaseings.
*


マルコフなモデルに比べると良いです（例えば、「役名：改行、台詞」のようになっています）が、まだ「英単語っぽくないなにか」が含まれていたりして不満足な気がします。

実は素朴な RNN には色々な問題があることが知られています：
- **勾配爆発**や**勾配消失**
- それに関連し、長期記憶の消失

以下では、これらの問題を部分的に解決するとされている2つの定番RNNを紹介します。

### 長・短期記憶(Long Short-Term Memory, LSTM)

まず初めに紹介するのは **長・短期記憶(Long Short-Term Memory, LSTM)** と呼ばれるユニットです。元論文は[こちら](https://www.mitpressjournals.org/doi/10.1162/neco.1997.9.8.1735)。LSTMは ${\bf h}_t$ に加えて、 **内部メモリー状態** ${\bf c}_t$ を導入し、このメモリ操作：「メモリ忘却、メモリ入力、出力更新」を組み込んだユニットです。このメモリ構造はチューリング機械の構造に似ています。

#### ゲートの追加
メモリの追加に加えて重要なのは **ゲート** と呼ばれるベクトルを用いた特徴量処理のメカニズムです。ゲートベクトル ${\bf g}$ とし、入力ベクトル ${\bf x}$ としたとき、ゲートによる処理は **成分毎の積(アダマール積)** $\odot$ をもちいて

$$
{\bf x} \odot {\bf g} = [x_1 g_1, x_2 g_2, \dots]
$$

と表されます。単なる成分ごとの掛け算なのですが、
- ${\bf g}$ の各成分が何らかの特徴量になっていて
- 成分積を取ることで その各成分を強調したり、減衰させたりする

役割を持たせたい、という気持ちを込めて、今回は絵で書く際は

![alt](gate.jpg)

のように、ベクトルの各成分を電気回路に模して描き、ゲートベクトルはゲート処理の大きさの度合いを表す「スイッチ」のように表現してみます。ここで重要なのは、ゲートベクトルの値はネットワーク内で訓練によって獲得されるという点です。訓練の目的に応じて、何らかの入力クエリから ${\bf g}$ は決定されるような構造をしています。
> ゲートの考え方は、**クエリによって注目すべき場所を変える** という思想が、後で紹介する**注意機構(attention mechanism)** と近い考え方をしていることがわかります。

この記法を用いると LSTM(を含んだ全体のネットワーク) は以下のように描けます：


![alt](rnn4.jpg)

sigmoid関数のマークから点線が各ゲートに向かって描かれていますが、それはsigmoid出力ベクトルをゲートベクトルに使うと言う意味で、sigmoidに入ってくるそれぞれのベクトルがそれぞれのゲートの処理を制御するための入力クエリとなります。素朴なLSTMは図で示してあるように、3種類のゲートと1つの入力処理（唯一 `tanh` が活性化関数に使われている部分）、出力ベクトルとメモリベクトル、から成ります。それぞれのゲートは
* ${\bf g^{forget}}$：**忘却ゲート** と呼ばれ、現在の時刻のクエリ(${\bf h}_t, {\bf e}_t$)に応じて、前時刻のメモリ ${\bf c}_{t-1}$ のうちどの成分を忘れるべきかを決定する
* ${\bf g^{input}}$：**入力ゲート** と呼ばれ、現在の時刻のクエリ(${\bf h}_t, {\bf e}_t$)に応じて、`tanh`による入力処理の成分のうちどれをメモリに書き込むべきかを決定する
* ${\bf g^{output}}$：**出力ゲート** と呼ばれ、現在の時刻のクエリ(${\bf h}_t, {\bf e}_t$)に応じて、メモリ ${\bf c}_t$ のうちどの成分を呼び出せばよいかを決定する

といった働きをしていると期待されます。
> これらのゲート処理は計算機の実装を思い起こさせます。実際、実はRNNを用いると、任意のアルゴリズムの実装が原理的には可能（たった一層でもチューリング完全）なのですが、これを凡例から**学習で獲得させる**となると、LSTMだけだとうまく行かないことが知られています。これはメモリの読み書きを「ベクトルの成分」で実装しているため、メモリの処理で重要な情報が上書きされやすいせいだと考えられます。後の節#で、アルゴリズム学習に特化した、より改善されたネットワーク構造を説明します。

具体的にはLSTMによる ${\color{red}{f_{rnn}}}({\bf e}_t)$ は以下で定義されます：

$$
{\color{red}{f_{rnn}}}({\bf e}_t) := {\bf h}_t
\left\{ \begin{array}{ll}
{\bf i}_t = \tanh ({\color{red}{W_{e}}} {\bf e}_t + {\color{red}{W_{h}}}{\bf h}_{t-1} + {\color{red}{{\bf b}}}  )  & {\text{Input}}
\\
{\bf c}_t = {\bf c}_{t-1} \odot \underbrace{{\bf \sigma}_{sigmoid}({\color{red}{W_{fe}}{\bf e}_t + {\color{red}{W_{fh}} {\bf h}_{t-1}  }} + {\color{red}{{\bf b}_f}})}_{{\bf g^{forget}}} + {\bf i}_t \odot \underbrace{{\bf \sigma}_{sigmoid}({\color{red}{W_{ie}}{\bf e}_t + {\color{red}{W_{ih}} {\bf h}_{t-1}  }}+ {\color{red}{{\bf b}_i}})}_{\bf g^{input}} & {\text{Memory Cell update}}
\\
{\bf h}_t = \tanh({\bf c}_t) \odot \underbrace{{\bf \sigma}_{sigmoid}({\color{red}{W_{oe}}{\bf e}_t + {\color{red}{W_{oh}} {\bf h}_{t-1}  }}+ {\color{red}{{\bf b}_o}})}_{\bf g^{output}}  & {\text{Output \& h-update}}
\end{array} \right.
$$



####  LSTMで訓練する
`MyRNN`の構造は同じで、上で`RNN_layer=SimpleRNN`だったところをLSTMに変えるだけです。さっきの`train_RNN()`で`RNN_layer=LSTM`のオプションを付けると勝手に"RNN_test"で始まるファイル名で訓練済みモデルのパラメータが保存されます。

In [8]:
%%time
loss_averages = train_RNN(RNN_layer = LSTM, epoch_size=15)

CPU times: user 17.9 s, sys: 2.93 s, total: 20.9 s
Wall time: 27.8 s


GPUで実行すると、tensorflowのデフォルト設定だと[cuDNN](https://developer.nvidia.com/cudnn)をうまく使ってくれる実装にしてあるようで、`SimpleRNN`よりも構造が複雑なくせに訓練が終わるのが早いです。（CPUだとこちらのほうが遅い？）

#### LSTMでシェイクスピア台詞生成
こちらも、同じ関数でオプションを変えることで上で訓練したLSTMを使った台詞生成が可能です：

In [9]:
model=load_RNN(RNN_layer = LSTM)

for _ in range(5):
    model.sample_from("A", num_string=100)

ATHULEY:
Very by the gane of him.
*
Ad
By this, in my soply ragainio.
*
Ad
Ischear yours!
*
ANLEST:
I'll be pay thee, or it: and then.
*
AMERLIUS:
Ay, give me as thing it be dase.
*


たまに変な単語が現れたりしますが、かなり自然な台詞っぽく見えるようになりました。ただし、お気付きの通り文法は変なままです。文法についてはもう少し工夫した訓練をしないと無理かもしれません。
### ゲート付き再帰ユニット(Gated Recurrent Unit, GRU)
GRUは近年提案されたユニットです([arXiv:1406.1078](https://arxiv.org/abs/1406.1078))。LSTMと同様、メモリとゲートの導入したユニットです。まずは全体図をお見せします：

![alt](rnn3.jpg)

GRUでは、LSTMの軽量化が図られているのがわかります。実際、出力ベクトル ${\bf h}_t$ は最早ユニット内で再帰せず、${\bf c}_t$ だけが再帰構造を持っています。加えて、LSTMには3つあったゲートが2つに減らされています。それぞれ
* ${\bf g^{reset}}$：**リセットゲート**、現在の入力 ${\bf e}_t$ と一時刻前のセル ${\bf c}_{t-1}$ をクエリとし、一時刻前のセル ${\bf c}_{t-1}$ をどれくらい現在の入力に入れるべきかを決定する
* ${\bf g^{update}}$：**アップデートゲート**、現在の入力 ${\bf e}_t$ と一時刻前のセル ${\bf c}_{t-1}$ をクエリとし、一時刻前のセル ${\bf c}_{t-1}$ と 現在の出力 ${\bf h}_t$ の混合の仕方を決定する。混合されたベクトルが現在のセル ${\bf c}_t$ となる

ものすごく大雑把に言うと、LSTMのゲートとGRUのゲートの対応は

.|LSTMのゲート|GRUのゲート
---|:---:|:---:
出力に関する処理|${\bf g^{forget}, g^{output}}$|${\bf g^{reset}}$
メモリに関する処理|${\bf g^{forget}, g^{input}}$|${\bf g^{update}}$

のようになっています。ゲートが減った分、LSTMよりも軽いユニットになっています。式をかくと以下のようになっています：


$$
{\color{red}{f_{rnn}}}({\bf e}_t) := {\bf h}_t
\left\{ \begin{array}{ll}
{\bf h}_t = \tanh ({\color{red}{W_{he}}} {\bf e}_t + {\color{red}{W_{hrc}}} [\underbrace{{\bf \sigma}_{sigmoid}({\color{red}{W_{re}}{\bf e}_t + {\color{red}{W_{rh}} {\bf c}_{t-1}  }})}_{{\bf g^{reset}}} \odot {\bf c}_{t-1}] )  & {\text{Output}}
\\
{\bf c}_t = \underbrace{{\bf \sigma}_{sigmoid}({\color{red}{W_{ue}}{\bf e}_t + {\color{red}{W_{uh}} {\bf c}_{t-1}  }})}_{{\bf g^{update}}}\odot({\bf c}_{t-1} - {\bf h}_t) + {\bf h}_t = {\bf g^{update}} \odot {\bf c}_{t-1} + [{\bf 1-  g^{update}}] \odot {\bf h}_t & \text{Cell update}
\end{array} \right.
$$

####  GRUで訓練する
同上です。

In [10]:
%%time
loss_averages = train_RNN(RNN_layer = GRU, epoch_size=15)

CPU times: user 16.8 s, sys: 2.68 s, total: 19.4 s
Wall time: 25 s


LSTMより訓練完了が、やや早いのがわかります。

#### GRUでシェイクスピア台詞生成

In [11]:
model=load_RNN(RNN_layer = GRU)

for _ in range(5):
    model.sample_from("A", num_string=100)

AND UpERENDurga.
*
Able done: there's no mercy.
*
AND Lord Angelo!
*
AND Kpepprest: the dobe conselss will thrie bands.
*
AQTHAMENLUS:
Are you there, the more has on the man as he.
*


このようにGRUでもそれなりにシェイクスピアっぽく見える文章生成を作れることがわかります。

### Licence in this subsection
In this notebook, the definitions of

* class method: `Generator.sample_from()`
* function: `get_data_dicts()`
* class: `MyRNN`
* function: `load_RNN`

include codes derived from
https://www.tensorflow.org/tutorials/text/text_generation
which is licenced under [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0). For details, see their [Site Policies](https://developers.google.com/terms/site-policies).



